<a href="https://colab.research.google.com/github/betty9895/ml-0716/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

f = zipfile.ZipFile("drive/My Drive/0724-news/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/0724-news/chinese_news_test.zip")
f.extractall("test")

In [ ]:
import glob
import os
import pandas as pd

def getdf(targetdn):
  datas = {
      "path":[],
      "label":[]
  }
  base=""
  p = os.path.join(base, targetdn, "*" )
  for dn in glob.glob(p):
    name = os.path.split(dn)[-1]
    fn = os.path.join(dn, "*.[tT][xX][tT]")
    fns = glob.glob(fn)
    datas["label"] = datas["label"] + [name] * len(fns)
    datas["path"] = datas["path"] +fns
  return pd.DataFrame(datas)

In [ ]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")

In [ ]:
def read_content(fn):
  with open(fn ,"r", encoding= "utf8") as f:
    content = f.read()
  return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)

In [ ]:
test_df

,path,label,content
0,test/chinese_news_test/軍事/8240.txt,軍事,美海軍成功演示飛機在空中控制無人機(附圖) \n\n--------------------...
1,test/chinese_news_test/軍事/8249.txt,軍事,\n軍事評論：“巴格達之戰”考驗美軍巷戰武器裝備(2) \n\n--------------...
2,test/chinese_news_test/軍事/8245.txt,軍事,軍事評論：美英聯軍地面作戰裝備特點分析(2) \n\n-------------------...
3,test/chinese_news_test/軍事/8244.txt,軍事,在伊拉克戰爭中使用的空襲精確制導彈藥(1) \n\n--------------------...
4,test/chinese_news_test/軍事/8247.txt,軍事,軍事評論：從巷戰或地道戰看單兵夜視裝備的發展 \n\n-------------------...
...,...,...,...
96,test/chinese_news_test/政治/9505.TXT,政治,楊主席提出促進中拉關係的５項建議\n新華社蒙得維的亞５月２３日電正在這...
97,test/chinese_news_test/政治/9501.TXT,政治,阿拉伯外長會議閉幕\n新華社巴格達５月２４日電為期２天的阿拉伯外長會\...
98,test/chinese_news_test/政治/9500.TXT,政治,外事簡訊\n新華社北京５月２４日電外事簡訊\n 全國人大常委會副...
99,test/chinese_news_test/政治/9497.TXT,政治,第３次安第斯集團首腦會議閉幕\n新華社庫斯科（祕魯）５月２３日電第３次...


In [ ]:
u = train_df["label"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
y_train = train_df["label"].replace(trans)
y_test = test_df["label"].replace(trans)
# y_train
# y_test

In [ ]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")

def news_cut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(news_cut)
x_test = test_df["content"].apply(news_cut)
# x_test

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.513 seconds.
Prefix dict has been built successfully.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [ ]:
test = input("輸入一首新聞:")
count = vec.transform([news_cut(test)])
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0]
for name, p in zip(u, proba):
    print(name, "的機率:", p)
print("答案:", trans_r[answer])

輸入一首新聞:針對馬英九質疑三倍券，總統蔡英文23日回應，三倍券的領取過程，不論超商或郵局其實程序都相當方便，全台灣這麼多人去領，「所以我要跟馬前總統說，不要害怕，趕快去，一點都不困難」。  蘇貞昌24日受訪也表示，看起來他比蔡總統還不勇敢，他很多觀念、做法，甚至好惡跟我們都不太一樣。年初國內口罩不夠用，他卻說要捐給中國，當我們提出什麼做法，他老是站在負面觀點。馬前總統當然可以有他的觀點，這點我們尊重，但三倍券確實好用，希望他這次能一起鼓勵民眾使用三倍券，不必怕！  馬英九日前出席「中華民國國際法學會圓桌論壇：南海爭端與我國之因應」被問到是否已領三倍券？他表示，自己還沒領，覺得三倍券程序很複雜，有點怕怕。媒體也詢問他問到領了之後要怎麼用？馬英九再次強調，還沒想到這個，因為覺得很複雜，有點怕怕，不曉得想幹嘛。
軍事 的機率: 1.3980649946286127e-07
交通 的機率: 3.007807121818094e-35
計算機 的機率: 1.6989980997845924e-05
環境 的機率: 6.59665369893308e-22
藝術 的機率: 4.960138889866007e-20
醫藥 的機率: 5.183794669588053e-31
教育 的機率: 2.694195342205486e-15
經濟 的機率: 0.999982870212481
體育 的機率: 6.85342282688877e-23
政治 的機率: 5.122084515242893e-15
答案: 經濟
